# Because nobody wants to wait for pandas.

In [ ]:
import numpy as np
import scipy.interpolate
import xarray as xr

from fair import FAIR
from fair.interface import initialise, fill
from fair.io import read_properties

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
species, properties = read_properties()

In [ ]:
f = FAIR(ch4_method='thornhill2021')
f.define_time(1750, 2500, 1)
f.define_configs(['unspecified'])
f.define_scenarios(scenarios)
f.define_species(species, properties)

In [ ]:
f.allocate()
f.fill_from_rcmip()

In [ ]:
f.emissions.loc[dict(specie='CH4')]

In [ ]:
f.emissions.to_netcdf('../data/rcmip/ssp_emissions_fair2.1.nc')

In [ ]:
scenarios = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

In [ ]:
species = [
    'CO2 FFI', 'CO2 AFOLU', 'CO2', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1202', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C6F14',
    'SF6',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-227ea', 'HFC-23', 'HFC-245fa', 'HFC-32', 
    'HFC-4310mee', 'Solar', 'Volcanic', 'Aerosol-radiation interactions',
    'Aerosol-cloud interactions', 'Ozone', 'Light absorbing particles on snow and ice',
    'Land use', 'Stratospheric water vapour', 'Equivalent effective stratospheric chlorine',
    'Contrails'#, 'NOx aviation'
]

In [ ]:
species, properties = read_properties(species=species)

In [ ]:
f = FAIR(ch4_method='thornhill2021')
f.define_time(1765, 2500, 1)
f.define_configs(['unspecified'])
f.define_scenarios(scenarios)
f.define_species(species, properties)

In [ ]:
f.allocate()
f.fill_from_rcmip()

In [ ]:
rcp_aviation_years = [1765, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2005, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]
rcp26_aviation_nox_data = [0, 0, 0.088, 0.190, 0.410, 0.889, 1.335, 2.045, 2.793, 3.162, 3.513, 4.156, 3.598, 4.039, 4.108, 2.664, 0.932, 0.343, 0.335, 0.337]
rcp45_aviation_nox_data = [0, 0, 0.088, 0.190, 0.410, 0.889, 1.335, 2.045, 2.793, 2.946, 3.111, 3.440, 4.030, 4.615, 5.194, 5.676, 6.123, 6.534, 7.229, 7.923]
rcp60_aviation_nox_data = [0, 0, 0.088, 0.190, 0.410, 0.889, 1.335, 2.045, 2.793, 2.955, 3.077, 3.395, 3.892, 4.486, 4.886, 5.042, 4.946, 4.565, 4.258, 3.865]
rcp85_aviation_nox_data = [0, 0, 0.088, 0.190, 0.410, 0.889, 1.335, 2.045, 2.793, 3.139, 3.486, 4.185, 4.937, 5.767, 6.543, 7.679, 8.848, 9.910,10.938,11.945]

interpolator = scipy.interpolate.interp1d(rcp_aviation_years, rcp26_aviation_nox_data)
rcp26_aviation_nox_emissions = np.zeros(735)
rcp26_aviation_nox_emissions[:335] = interpolator(np.arange(1765.5, 2100))
rcp26_aviation_nox_emissions[335:] = rcp26_aviation_nox_emissions[334]

interpolator = scipy.interpolate.interp1d(rcp_aviation_years, rcp45_aviation_nox_data)
rcp45_aviation_nox_emissions = np.zeros(735)
rcp45_aviation_nox_emissions[:335] = interpolator(np.arange(1765.5, 2100))
rcp45_aviation_nox_emissions[335:] = rcp45_aviation_nox_emissions[334]

interpolator = scipy.interpolate.interp1d(rcp_aviation_years, rcp60_aviation_nox_data)
rcp60_aviation_nox_emissions = np.zeros(735)
rcp60_aviation_nox_emissions[:335] = interpolator(np.arange(1765.5, 2100))
rcp60_aviation_nox_emissions[335:] = rcp60_aviation_nox_emissions[334]

interpolator = scipy.interpolate.interp1d(rcp_aviation_years, rcp85_aviation_nox_data)
rcp85_aviation_nox_emissions = np.zeros(735)
rcp85_aviation_nox_emissions[:335] = interpolator(np.arange(1765.5, 2100))
rcp85_aviation_nox_emissions[335:] = rcp85_aviation_nox_emissions[334]

In [ ]:
fnew = xr.DataArray(
    np.ones(
        (
            f._n_timepoints,
            f._n_scenarios,
            f._n_configs,
            f._n_species + 1,
        )
    )
    * np.nan,
    coords=(f.timepoints, f.scenarios, f.configs, species + ['NOx aviation']),
    dims=("timepoints", "scenario", "config", "specie"),
)
fnew[:, :, :, :-1] = f.emissions.data
fnew[:, 0, 0, -1] = rcp26_aviation_nox_emissions
fnew[:, 1, 0, -1] = rcp45_aviation_nox_emissions
fnew[:, 2, 0, -1] = rcp60_aviation_nox_emissions
fnew[:, 3, 0, -1] = rcp85_aviation_nox_emissions

In [ ]:
fnew

In [ ]:
fnew.loc[dict(specie='CH4', scenario='rcp45')]

In [ ]:
fnew.to_netcdf('../data/rcmip/rcp_emissions_fair2.1.nc')